In [ ]:
!pip install -q langchain chroma chromadb sentence-transformers pypdf langchain_community llama-parse llama-index chainlit langchain_experimental


In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings


In [ ]:
from llama_parse import LlamaParse


In [ ]:
parser = LlamaParse(
    api_key="llx-4xS3dvyywg9NC1DjOBwXC5meYfXDV3PFURMz31egW9N0Iw8v",
    result_type="markdown"
)



In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
documents = parser.load_data(["/content/ADHD.pdf", "/content/India_0.pdf" , "/content/Mental Health Support Numbers.pdf" , "/content/generalized_anxiety_disorder.pdf" , "/content/bipolar guide en.pdf" , "/content/adhd-in-adults.pdf" , "/content/Bipolar-disorder-YHIM.pdf" , "/content/OH-195.20-Anxiety.pdf"])

Started parsing the file under job_id 47903d08-dc87-4203-9aa6-9282c1edbba4
Started parsing the file under job_id 1afadcdf-f639-46e1-9fc4-4aaf40cf8832
Started parsing the file under job_id 9b67eae3-b8f5-4329-aab6-942ea0f8ec3a
Started parsing the file under job_id 9fc8bc1e-313f-4be8-a038-3d325cbe8443
Started parsing the file under job_id 07ed9eef-3e43-4a0d-acc4-9be32812c053
Started parsing the file under job_id 485ee903-880e-4fcf-92e2-51ab611922eb
Started parsing the file under job_id 19b8a9e1-464e-49c9-9e84-4b39fdaf70bb
Started parsing the file under job_id baa4e2e0-3e17-4471-9155-db4150efbc83


In [ ]:
docs = documents.__str__()

In [ ]:
model_name = "NeuML/pubmedbert-base-embeddings"
# model_kwargs = {'device': 'cpu'}
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
hf_embeddings = HuggingFaceEmbeddings()
text_splitter = SemanticChunker(hf_embeddings)



In [ ]:
chunks = text_splitter.create_documents([docs])

In [ ]:
vectorstore = Chroma.from_documents(chunks , hf_embeddings , persist_directory="/content/chroma1")

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs = {'k':5})

In [ ]:
query = "what is adhd"
search = vectorstore.similarity_search(query)

In [ ]:
!nvidia-smi


Wed Mar  6 03:55:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              27W /  70W |   1991MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
retrieved = retriever.get_relevant_documents(query)

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [ ]:
llm = LlamaCpp(
    model_path = '/content/drive/MyDrive/BioMistral-7B.Q4_K_M.gguf' ,
    temperature = 0.3,
    top_p = 1,
    max_tokens = 2048 ,
    verbose=True,
    n_gpu_layers = -1,
    n_batch = 512 ,
    n_ctx=32768 ,
    callback_manager=callback_manager,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = """
<|context|>
You are a medical AI assistant trained on extensive medical data.
Please provide accurate and concise responses.
</s>
<|user|>
{query}
</s>
 <|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response = rag_chain.invoke("Could you provide me with some suicide helpline numbers for individuals in India who may be in distress?")

In [ ]:
response

"I'm sorry to hear that. Here are some suicide prevention helplines in India that could help:1. National Mental Health Helpline (24x7): 181953819192. Samaritans India (24x7): +9199263076763. Parivartan (24x7): +9199263076764. Kutu (24x7): +9199263076765. Jeevan (24x7): +9199263076766. Sneha (24x7): +9199263076767. Pratheek (24x7): +9199263076768. Pratyaksha (24x7): +9199263076769. Pratishtha (24x7): +9199263076770. Sangath (24x7): +9199263076771. Pradip (24x7): +9199263076772. Vartman (24x7): +9199263076773. Aarogya Manthan (24x7): +9199263076774. Sanjivani (24x7): +9199263076775. Pratipak (24x7): +9199263076776."

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ",result)